In [1]:
!wget -q https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/1/gesture_recognizer.task
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [16]:
import cv2
from picamera2 import Picamera2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
import time
import sys, math

In [ ]:
def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])



base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=False,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


with Picamera2() as picam:
    picam.configure(picam.create_video_configuration(main={"format": 'RGB888', "size": (1280, 720)}))
    picam.start()

    while True:
        start_time = time.time()
        clear_output(wait=True)    
        image = picam.capture_array("main")
        image = cv2.flip(image, 1)

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        detection_result = detector.detect(mp_image)

        try:
            transformation_matrix = detection_result.facial_transformation_matrixes[0]
            rotation_angles = rotation_matrix_to_euler_angles(transformation_matrix)
            
            yaw_angle = np.degrees(rotation_angles[1])
            
            if yaw_angle < -20:
                print("👈")
            elif yaw_angle > 20:
                print("👉")
            else:
                print("🫵")

        except Exception as e:
            display("Error: ", e)
            continue
        
        display("FPS: ", 1.0 / (time.time() - start_time))